In [2]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from typing import Literal
import os

print("✅ All imports successful")

c:\Users\HP\Desktop\mywork\NOTEBOOKS\working_with_llms\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All imports successful


In [3]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded")

✅ API key loaded


In [4]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-5-nano",
    temperature=0.5,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-5-nano


In [5]:

file_path = "psych-pdfs"

# Check if file exists
if not os.path.exists(file_path):
    print(f"⚠️ File not found: {file_path}")
    print("Please update the file_path variable with your PDF file.")
    print("\nFor this demo, we'll create sample documents instead...")
    
    
# Load the PDF
loader = PyPDFDirectoryLoader(file_path)
pages = []
    
# Load pages (async loading)
async for page in loader.alazy_load():
    pages.append(page)
    
print(f"✅ Loaded {len(pages)} pages from PDF")

✅ Loaded 1107 pages from PDF


In [6]:
# Create text splitter (Module 2 knowledge!)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,      # Characters per chunk
    chunk_overlap=200     # Overlap to preserve context
)

# Split documents
doc_splits = text_splitter.split_documents(pages)

print(f"✅ Created {len(doc_splits)} chunks")
print(f"\nSample chunk:")
print(f"{doc_splits[0].page_content[:200]}...")

✅ Created 1417 chunks

Sample chunk:
Discovering Psychology Series 
 
 
Abnormal Psychology 
2nd edition 
 
 
Alexis Bridley, Ph.D. 
Lee W. Daffin Jr., Ph.D. 
Washington State University 
 
 
Version 2.00 
August 2020 
 
 
Contact Inform...


In [7]:
# Initialize embeddings (using OpenAI)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=openai_api_key
)

print("✅ Embeddings model initialized")

✅ Embeddings model initialized


In [14]:
# Create Chroma vector store
chroma_path = "./chroma_db_agentic_rag"

# Create vector store from documents
vectorstore = Chroma(
    collection_name="agentic_rag_docs",
    persist_directory=chroma_path,
    embedding_function=embeddings
)

# Add documents
vectorstore.add_documents(documents=doc_splits)

print(f"✅ Vector store created with {len(doc_splits)} chunks")
print(f"   Persisted to: {chroma_path}")

✅ Vector store created with 1417 chunks
   Persisted to: ./chroma_db_agentic_rag


In [15]:
@tool
def retrieve_documents(query: str) -> str:
    """
    Search for relevant documents in the knowledge base.
    
    Use this tool when you need information from the document collection
    to answer the user's question. Do NOT use this for:
    - General knowledge questions
    - Greetings or small talk
    - 
    
    Args:
        query: The search query describing what information is needed
        
    Returns:
        Relevant document excerpts that can help answer the question
    """
    # Use MMR (Maximum Marginal Relevance) for diverse results
    retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 5, "fetch_k": 10}
    )
    
    # Retrieve documents
    results = retriever.invoke(query)
    
    if not results:
        return "No relevant documents found."
    
    # Format results
    formatted_chunks = []
    for i, doc in enumerate(results):
        source = os.path.basename(doc.metadata.get("source", "Unknown source"))
        page = doc.metadata.get("page", "Unknown page")

        formatted_chunks.append(
            f"Document {i + 1}:\n"
            f"Source: {source} | Page: {page}\n\n"
            f"{doc.page_content}"
        )
    
    return "\n\n---\n\n".join(formatted_chunks)

print("✅ Retrieval tool created")

✅ Retrieval tool created


In [50]:
# Test tool directly
test_result = retrieve_documents.invoke({"query": "What are the Big Five?"})
print(f"Tool result (first 300 chars):\n{test_result[:300]}...")

Tool result (first 300 chars):
Document 1:
Source: personality.pdf | Page: 64

Five trait taxonomy: History, measurement, and conceptual issues. In O. P . John, R. 
W. Robins, & L. A. Pervin (Eds.), Handbook of personality: Theory and research (pp. 
114-158). New Y ork, NY: Guilford Press.
Judge, T.A., Higgins, C.A., Thoresen, C....


In [25]:
system_prompt = SystemMessage(content="""You are a psychology academic assistant with access to a document retrieval tool.

RETRIEVAL CRITICAL DECISION RULES:

DO NOT retrieve for:
- Greetings: "Hello", "Hi", "How are you"
- Questions about your capabilities: "What can you help with?", "What do you do?"
- General knowledge
- Definitions of well-known concepts
- Casual conversation: "Thank you", "Goodbye"

DO retrieve for:
- Questions asking for specific information that would be in documents
- Requests for facts, definitions, or explanations about specialized topics
- Any question where citing sources would improve the answer

Rule of thumb: If the user is asking for information (not just chatting), retrieve first.

When you retrieve documents, Cite source and page number in your answer. Base the answer strictly on retrieved contentIf documents don't contain the answer, say so.
""")

print("✅ System prompt configured")

✅ System prompt configured


In [26]:
# Bind tool to LLM
tools = [retrieve_documents]
llm_with_tools = llm.bind_tools(tools)

def assistant(state: MessagesState) -> dict:
    """
    Assistant node - decides whether to retrieve or answer directly.
    """
    messages = [system_prompt] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """
    Decide whether to call tools or finish.
    """
    last_message = state["messages"][-1]
    
    if last_message.tool_calls:
        return "tools"
    return "__end__"

print("✅ Agent nodes defined")

✅ Agent nodes defined


In [27]:
# Build graph
builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)
builder.add_edge("tools", "assistant")

# Add memory
memory = MemorySaver()
agent = builder.compile(checkpointer=memory)

print("✅ Agentic RAG system compiled")

✅ Agentic RAG system compiled


In [28]:
def query_agent(user_input: str, thread_id: str = "default_session"):
    """
    Improved query function with clearer output.
    """
    print(f"\n{'='*70}")
    print(f"👤 User: {user_input}")
    print(f"{'='*70}\n")

    result = agent.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"thread_id": thread_id}}
    )

    # Check what happened
    used_retrieval = False
    final_answer = None

    for message in result["messages"]:
        if isinstance(message, AIMessage):
            if message.tool_calls:
                used_retrieval = True
                print(f"🔍 Agent: [Calling retrieval tool...]")
            if message.content and not message.tool_calls:
                final_answer = message.content

    # Always print final answer
    if final_answer:
        print(f"🤖 Agent: {final_answer}")
    else:
        print(f"⚠️ No response generated after retrieval!")

    # Summary
    print(f"\n📊 Decision: {'USED RETRIEVAL' if used_retrieval else 'ANSWERED DIRECTLY'}")
    print(f"{'='*70}\n")



In [20]:
# Query 1: Retrieval Needed 
query_agent("What are the four main processes that are involved in observational learning?")


👤 User: What are the four main processes that are involved in observational learning?

🔍 Agent: [Calling retrieval tool...]
🤖 Agent: The four main processes are:
- Attention
- Retention (memory)
- Motor reproduction
- Reinforcement (motivation)

Source: Behavioral Psych.pdf, page 41.

📊 Decision: USED RETRIEVAL



In [ ]:
# Query 2: Greeting (No retrieval)
query_agent("Hello")


👤 User: Hello

🤖 Agent: Hi there! How can I help with psychology today? I can discuss theories, design or critique studies, analyze data, summarize research, or help locate relevant sources if you’d like me to fetch documents.

📊 Decision: ANSWERED DIRECTLY



In [23]:
# Query 3: Retrieval Needed (Specific process)
query_agent("Functioning of the Nervous system")


👤 User: Functioning of the Nervous system

🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🤖 Agent: Here’s a concise overview of the functioning of the nervous system with source citations.

- Divisions and core roles
  - CNS vs PNS: Central nervous system = brain and spinal cord; Peripheral nervous system = all nerves outside the CNS that connect to sensory organs, muscles, glands. (NeuroPsych.pdf, p.4)
  - The CNS acts as the control center; the PNS relays information between the CNS and the rest of the body. (NeuroPsych.pdf, p.4)

- Subdivisions of the PNS
  - Somatic nervous system: controls voluntary movement and carries sensory information to the CNS. Autonomic nervous system: regulates internal organs, glands, and blood vessels; has sympathetic and parasympathetic branches. (Abnormal-Psychology-2nd-Edition.pdf, p.76)

- Sympathetic and parasympathetic systems
  - Sympathetic = ar

In [29]:
# Query 4: General Knowledge (Coding task)
query_agent("What is the capital of France?")


👤 User: What is the capital of France?

🤖 Agent: Paris.

📊 Decision: ANSWERED DIRECTLY



In [31]:
# Query 5: Retrieval Needed (Comparison)
query_agent("Functions of neuropsychologists")


👤 User: Functions of neuropsychologists

🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🤖 Agent: Here are the main functions of neuropsychologists, drawn from the NeuroPsych.pdf materials:

- Clinical assessment and diagnosis: perform neuropsychological evaluations to characterize cognitive strengths/weaknesses, differentiate conditions, and inform diagnosis and treatment planning. (NeuroPsych.pdf, page 9)

- Cognitive testing across core domains: administer and interpret tests that measure domains such as memory (short/long term), attention, executive function, language, visuospatial skills, processing speed, motor skills, and related factors. (NeuroPsych.pdf, page 9; Major domains listed in 1.6, page 11)

- Rehabilitation planning and intervention: design and implement cognitive rehabilitation programs and compensatory strategies to improve or adapt daily functioning. (NeuroPsych.pdf, page 9)

- Prognosis, treatment planning, and monitoring: use test resul

In [33]:
# Query 6: General Knowledge (Simple Math)
query_agent("What is 2*50?")


👤 User: What is 2*50?

🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🤖 Agent: 100

📊 Decision: USED RETRIEVAL



In [34]:
# Query 7: Retrieval Needed 
query_agent("How do neurons relay information to each other")


👤 User: How do neurons relay information to each other

🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🤖 Agent: Neurons relay information through both electrical signaling inside cells and chemical signaling between cells. Here’s how it works, step by step:

- Resting state: A neuron has a resting membrane potential (about -70 mV) maintained by ion pumps and leak channels.

- Dendritic input: Stimuli cause small voltage changes (graded potentials) in the dendrites and soma. If the combined input reaches a threshold, an action potential is triggered at the axon hillock.

- Action potential (electrical signal): 
  - Depolarization: Voltage-gated sodium channels open, sodium ions rush in, and the inside becomes more positive.
  - Peak and repolarization: Sodium channels close, voltage-gated potassium channels open, potassium exits, restoring negative charge.
  - Refractory period: The neuron briefly becomes less excitable, then returns to resting state. In myel

BRIEF REPORT 
1. Domain Choice

I chose "Cogent Topics in Psychology" as my domain because it resonates better with me as it is my discipline. It contains a clear distinction between general psychological knowledge and document-specific academic content.

2. Chunk Size Tuning

Initial tests with 800 tokens caused splitted answers. Increasing to 800 tokens with overlap improved semantic correlation and preserved important contextual relationships between concepts, resulting in more complete and accurate responses.

3. Retrieval Decisions

The agent made strong decisions for most queries. General questions like “What is Psychology?” correctly avoided retrieval, relying instead on the knowledge base.

4. What Worked Well

•	MMR retrieval reduced redundancy

•	Explicit system prompt improved decision accuracy

•	Including source and page metadata inclusion improved transparency

5. Areas for Improvement

•	Some borderline questions could benefit from confidence scoring to better determine whether retrieval is necessary

•	Adding a reranking step could further improve the relevance of retrieved passages

•	Long, multi-part psychology questions may require multi-hop retrieval to integrate information across multiple documents

